# ECG Signal Processing - Variant 2

In [ ]:
!pip install wfdb biosppy neurokit2

## 2. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import neurokit2 as nk

## 3. Parametry Symulacji

In [ ]:
# Czas trwania i częstotliwość próbkowania
duration = 10        # seconds
fs = 500             # Hz (sampling rate)

# Filtr pasmowy: dolna 1 Hz, górna 35 Hz
lowcut = 1.0         # Hz
highcut = 35.0       # Hz
order = 4     

 ## 4. Generacja i Filtrowanie Sygnału ECG

4.1. Symulacja sygnału ECG

In [ ]:
ecg_signal = nk.ecg_simulate(duration=duration, sampling_rate=fs)

4.2. Definicja filtru pasmowego

In [ ]:
def bandpass_filter(signal, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

4.3. Filtrowanie sygnału

In [ ]:
ecg_filtered = bandpass_filter(ecg_signal, lowcut, highcut, fs, order)

## 5. Wizualizacja Sygnału

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(ecg_signal, label='Raw ECG')
plt.plot(ecg_filtered, label='Filtered ECG (1-35 Hz)')
plt.title('Raw vs Filtered ECG Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude (a.u.)')
plt.legend()
plt.tight_layout()
plt.show()

## 6. Wykrywanie R-peaków

In [ ]:
signals, info = nk.ecg_process(ecg_filtered, sampling_rate=fs)
r_peaks = info['ECG_R_Peaks']

plt.figure(figsize=(12, 4))
plt.plot(ecg_filtered, label='Filtered ECG')
plt.plot(r_peaks, ecg_filtered[r_peaks], 'ro', label='R-peaks')
plt.title('R-peak Detection')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.legend()
plt.tight_layout()
plt.show()

 ## 7. Obliczanie Częstości Serca (Instantaneous Heart Rate)

In [ ]:

# Indeksy w próbkach -> czasy w sekundach
r_times = np.array(r_peaks) / fs
# RR-intervals w sekundach
rr_intervals = np.diff(r_times)
# HR instantaneous (bpm)
hr_inst = 60 / rr_intervals
# Czasy przypisane do HR jako środek przedziału RR
time_hr = r_times[:-1] + rr_intervals / 2

plt.figure(figsize=(12, 4))
plt.plot(time_hr, hr_inst, marker='o')
plt.title('Instantaneous Heart Rate over Time')
plt.xlabel('Time (s)')
plt.ylabel('Heart Rate (bpm)')
plt.grid(True)
plt.tight_layout()
plt.show()